In [1]:
# Libraries
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.linalg
import scipy.sparse as sp


importing NGSolve-6.2.2301


In [2]:
# Setting the geometry


geo = SplineGeometry()
# Setting global decomposition vertices (coarse mesh)
Points = [(0,0), (1,0), (2,0), 
          (2,1), (1,1), (0,1)]
# Setting edgle lables counter-clock wise 
bcs_edge = ["bottom0", "bottom1", "right", "top1", "top0", "left", "middle"]
# Labeling vertices V1,...,V6
for i, pnt in enumerate(Points):
    geo.AddPoint(*pnt, name = "V" + str(i))
# Labeling edges by specifying end points, neighbouring domains (counterclock-wise), label
geo.Append(["line", 0, 1], leftdomain=1, rightdomain=0, bc="bottom0")
geo.Append(["line", 1, 2], leftdomain=2, rightdomain=0, bc="bottom1")
geo.Append(["line", 2, 3], leftdomain=2, rightdomain=0, bc="right")
geo.Append(["line", 3, 4], leftdomain=2, rightdomain=0, bc="top1")
geo.Append(["line", 4, 5], leftdomain=1, rightdomain=0, bc="top0")
geo.Append(["line", 5, 0], leftdomain=1, rightdomain=0, bc="left")
geo.Append(["line", 1, 4], leftdomain=1, rightdomain=2, bc="middle")

# ngmesh = unit_square.GenerateMesh(maxh=0.1)
mesh = Mesh(geo.GenerateMesh(maxh = 0.1))
# Labeling subdomains in coarse mesh
mesh.ngmesh.SetMaterial(1,"omega0") 
mesh.ngmesh.SetMaterial(2,"omega1")
Draw(mesh)
print(mesh.GetMaterials()) # Subdomains
print(mesh.GetBoundaries()) # Edges
print(mesh.GetBBoundaries()) # Vertices
#input()

('omega0', 'omega1')
('bottom0', 'bottom1', 'right', 'top1', 'top0', 'left', 'middle')
('V0', 'V1', 'V2', 'V3', 'V4', 'V5')


In [4]:

# Basis functions per subdomains (constant for all subdomains)
bubble_modes = 2
edge_modes = 3

# Definition of Sobolev space, order of polynomial approximation can be increased
# The Dirichlet condition is imposed everywhere because it will be needed for the basis construction
# It does not effectively remove the boundary nodes
V = H1(mesh, order = 1, dirichlet = ".*")

# Saving in a vector the boundaries of the boundary, namely the vertices of the edges that are on the boundary
# This returns a vector of 0 for internal mesh vertices and 1 for edge mesh vertices (coarse vertices)
vertex_dofs = V.GetDofs(mesh.BBoundaries(".*")) 
#print(vertex_dofs)


all_edge_freedofs = BitArray(V.ndof)
all_edge_freedofs.Clear()
edge_freedofs = {}
for edge_name in mesh.GetBoundaries():
    free_dofs = V.GetDofs(mesh.Boundaries(edge_name))
    for i, b in enumerate(vertex_dofs):
        if b == 1:
            free_dofs[i] = 0
        edge_freedofs[edge_name] = free_dofs
    all_edge_freedofs = all_edge_freedofs | free_dofs

# all dofs on the skeleton without vertex dofs


u, v = V.TnT()

a = BilinearForm(V)
a += grad(u) * grad(v) * dx
a.Assemble()

m = BilinearForm(V)
m += u * v * dx
m.Assemble()


# in V.Freedofs we only have the interior dofs 
# since we set a dirichlet flag on the whole boundary 
gfu = GridFunction(V)
res = gfu.vec.CreateVector()

a_inv = a.mat.Inverse(V.FreeDofs())



**Extension operators**

We need an extension from $\Gamma$ to $\Omega$, which is obtained by combining the extensions of functions from $\partial\Omega_jj$ to $\Omega_j$.

For a given $\tau \in H^{1/2}(\partial \Omega_j)$, let $\tilde \tau \in H^1(\Omega_j)$ be any function satisfying $\tilde\tau_{\mid\partial\Omega_j}=\tau$.
Then, we indicate by $\tilde\tau_0 \in H_0^1(\Omega_j)$ the solution to

\begin{align} 
    \mathcal{A}_{j} (\tilde\tau_0, v)-(\kappa^2 \tilde\tau_0, v)_{\Omega_j} = -\left(\mathcal{A}_j(\tilde\tau,v)-(\kappa^2 \tilde\tau, v)_{\Omega_j}\right) \quad \forall  v\in  H_0^1(\Omega_j).
\end{align}

We characterize the $\mathcal{A}$-harmonic extension $E^{j}:H^{1/2}(\partial \Omega_j) \to H^1(\Omega_j)$ by setting 
    $E^{j}\tau := \tilde\tau+\tilde\tau_0$.

**Lemma** The extension operator $E^{j}:H^{1/2}(\partial \Omega_j) \to H^1(\Omega_j)$ is bounded, that is,
    \begin{equation}
        \|E^{j}\tau\|_{\mathcal{B}}\leq (1+\frac{1}{\beta^j})\|\tilde\tau\|_{\mathcal{B}},
    \end{equation}
    where $\tilde \tau\in H^1(\Omega_j)$ is any extension of $\tau\in \ H^{1/2}(\partial \Omega_j)$.
    

We note the following orthogonality relation, which is a crucial property for the construction of the ACMS  spaces: for all bubble functions $b_i^j \in H_0^1(\Omega_j)$, we have
\begin{align}
		\mathcal{A}_{j}(E^j \tau,b_i^j)-(\kappa^2 E^j \tau,b_i^j)_{\Omega_j}=0.
\end{align}

Assume that $e=\partial\Omega_j \cap \partial\Omega_i\in \mathcal{E}$ is a common edge of $\Omega_i$ and $\Omega_j$. Let $\tau \in H^{1/2}(\Gamma)$, which, by restriction, implies $\tau\in H^{1/2}(\partial \Omega_j)$ and $\tau\in H^{1/2}(\partial \Omega_i)$.


Extension on interface: $E^{\Gamma} : H^{1/2}(\Gamma) \to H^1_D(\Omega)$ by $(E^{\Gamma} \tau)_{\mid\Omega_j} = E^{j} \tau_{\mid \partial \Omega_j}$, for all $j = 1,\ldots,J$. 

Extension on edges: $E^{\Gamma} : H^{1/2}_{00}(e) \to H^1_D(\Omega)$ via $E^{\Gamma} \tau = E^{\Gamma} E_0^e\tau$, where $E_0^e: H^{1/2}_{00}(e)  \to H^{1/2}(\Gamma)$ denotes the extension by zero to the interface $\Gamma$.


**Edge basis**

Same for elliptic or Helmholtz case (changes the extension).

Let us consider $e\in\mathcal{E}$ and denote by $\partial_e$ the tangential derivative, i.e., differentiation along $e$.
We define the edge modes as solutions to the following weak formulation of the edge-Laplace eigenvalue problems: for each $e\in\mathcal{E}$, for $i \in \mathbb{N}$, find $(\tau^e_i,\lambda^e_i)\in H^{1/2}_{00}(e) \times\mathbb{R}$ such that
\begin{align}
(\partial_e \tau^e_i,\partial_e \eta)_e =\lambda^e_i ( \tau^e_i, \eta)_e \quad \text{for all } \eta\in H^{1/2}_{00}(e).
\end{align}

In [5]:
###############################################################
# EDGE BASIS

t = specialcf.tangential(2)

a_edge = BilinearForm(V)
a_edge += (grad(u)*t) * (grad(v)*t) * ds(skeleton = True) #, definedon=mesh.Boundaries("bottom"))

a_edge.Assemble()
a_edge_inv = a_edge.mat.Inverse(all_edge_freedofs)

m_edge = BilinearForm(V)
m_edge += u.Trace() * v.Trace() * ds()
m_edge.Assemble()

# edge_ev_evec = {}
# edge_basis = {}
def calc_edge_basis(basis):
    for edge_name in mesh.GetBoundaries():
        # edge_basis[edge_name] = []
        fd = edge_freedofs[edge_name]
        # print(fd)
        nd = sum(fd)

        A = Matrix(nd, nd)
        M = Matrix(nd, nd)

        dofs = []
        for i, b in enumerate(fd):
            if b == 1:
                dofs.append(i)



        for i in range(nd):
            for j in range(nd):
                A[i,j] = a_edge.mat[dofs[i], dofs[j]]
                M[i,j] = m_edge.mat[dofs[i], dofs[j]]

        # print(A)
        # print(M)

        ## ev numbering starts with zero!
        ev, evec = scipy.linalg.eigh(a=A, b=M, subset_by_index=[0, edge_modes-1])
        evec = evec.transpose()
        # edge_ev_evec[edge_name] = [ev, evec]

        for e in evec:
            gfu_extension = gfu.vec.CreateVector()
            gfu.vec[:] = 0
            for i in range(nd):
                gfu.vec[dofs[i]] = e[i]
            # Draw(gfu)
            res = a.mat * gfu.vec
            # gfu.vec.data += -ainv*res
            gfu_extension.data = gfu.vec - a_inv * res
            # Draw(gfu_extension, mesh, "extension")
            # input()
            # edge_basis[edge_name].append(gfu_extension)
            basis.Append(gfu_extension)

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )
used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


**Vertex basis**

**Helmholtz case:** For any $p\in\mathcal{V}$, let $\varphi_p: \Gamma\to \mathbb{R}$ denote a piecewise harmonic function, that is, $\Delta_e\varphi_{p\mid e}=0$ for all $e\in\mathcal{E}$, with $\Delta_e$ indicating the Laplace operator along the edge $e\in\mathcal{E}$, and $\varphi_p(q)=\delta_{p,q}$ for all $p,q\in\mathcal{V}$.
The vertex based space is then defined by linear combinations of corresponding  extensions:
\begin{align*}
V_{\mathcal{V}} = {\rm span}\{\, E^{\Gamma} \varphi_p \,:\, \ p\in\mathcal{V}\}.
\end{align*}
For our error analysis, we will employ the nodal interpolant
\begin{align}
I_{\mathcal{V}} v = \sum_{p\in \mathcal{V}} v(p) \varphi_p,
\end{align}
which is well-defined for functions $v:\overline{\Omega}\to\mathbb{C}$ that are continuous in all $p\in \mathcal{V}$. Moreover, note that the support of the vertex basis functions consists of all subdomains which share the vertex and is therefore local.

In [6]:
###############################################################
# VERTEX BASIS
def calc_vertex_basis(basis):
    for j,vertex_name in enumerate(mesh.GetBBoundaries()):
        # print(vertex_name)
        gfu_extension = gfu.vec.CreateVector()
        fd = V.GetDofs(mesh.BBoundaries(vertex_name)) 
        # print(fd)
        gfu.vec[:] = 0
        for i, b in enumerate(fd):
            if b == 1:
                gfu.vec[i] = 1

        # THIS IS JUST A LINEAR EXTENSION!!!!!!!
        # VERY EXPENSIVE AT THE MOMENT, FIND ALTERNATIVE!
        res = a_edge.mat * gfu.vec
        gfu_extension.data = gfu.vec - a_edge_inv * res

        res = a.mat * gfu_extension
        gfu_extension.data = gfu_extension - a_inv * res
        # vertex_basis[vertex_name] = gfu_extension
        basis.Append(gfu_extension)
        # gfu.vec.data = gfu_extension
        # Draw(gfu)
        # basis[j] = gfu_extension


**Bubble functions**


**Elliptic case:** Let us define the local bilinear form $\mathcal{A}{j}: H^1(\Omega_j) \times H^1(\Omega_j) \to \mathbb{R}$ with domain of integration $\Omega_j$ instead of $\Omega$.
Since $\mathcal{A}_{j}$ is symmetric, we can consider the eigenproblems: for $j=1,...,J$ and $i \in \mathbb{N}$, find $(b_i^j,\lambda_i^j)\in H_0^1(\Omega_j) \times \mathbb{R}$ such that
\begin{align}
	\mathcal{A}_{j}(b_i^j,v)= \lambda_i^j ( b_i^j,v)_{\Omega_j} \quad \forall v \in H_0^1(\Omega_j).
\end{align}

**Helmholtz case:** Let us define the local sesquilinear form $\mathcal{A}{j}: H^1(\Omega_j) \times H^1(\Omega_j) \to \mathbb{C}$ with domain of integration $\Omega_j$ instead of $\Omega$.
Since $\mathcal{A}_{j}$ is Hermitian, we can consider the eigenproblems: for $j=1,...,J$ and $i \in \mathbb{N}$, find $(b_i^j,\lambda_i^j)\in H)0^1(\Omega_j) \times \mathbb{R}$ such that
\begin{align}
	\mathcal{A}_{j}(b_i^j,v)= \lambda_i^j ( {\kappa^2} b_i^j,v)_{\Omega_j} \quad \forall v \in H_0^1(\Omega_j).
\end{align}

In [7]:
###############################################################
# BUBBLES

def calc_bubble_basis(basis):
    for mat_name in mesh.GetMaterials():
        # bubble_basis[mat_name] = []
        fd = V.GetDofs(mesh.Materials(mat_name)) & V.FreeDofs()
        
        nd = sum(fd)
        # print(fd)
        A = Matrix(nd, nd)
        M = Matrix(nd, nd)

        dofs = []
        for i, b in enumerate(fd):
            if b == 1:
                dofs.append(i)

        for i in range(nd):
            for j in range(nd):
                A[i,j] = a.mat[dofs[i], dofs[j]]
                M[i,j] = m.mat[dofs[i], dofs[j]]


        ## ev numbering starts with zero!
        ev, evec = scipy.linalg.eigh(a=A, b=M, subset_by_index=[0, bubble_modes-1])
        # print(ev)
        evec = evec.transpose()
        # edge_ev_evec[edge_name] = [ev, evec]
        for e in evec:
            bubble = gfu.vec.CreateVector()
            bubble[:] = 0
            for i in range(nd):
                bubble[dofs[i]] = e[i]
            # bubble_basis[mat_name].append(bubble)
            # basis[j] = bubble
            basis.Append(bubble)

In [ ]:





basis = MultiVector(gfu.vec, 0)

calc_vertex_basis(basis)
calc_edge_basis(basis)
calc_bubble_basis(basis)

num = len(basis)

u, v = V.TnT()

dom_bnd = "bottom0|bottom1|right|top1|top0|left"


kappa = 0
a = BilinearForm(V)
a += grad(u) * grad(v) * dx
a += -kappa * u * v * dx
a += 10**6 * u * v * ds(dom_bnd)
# a += u * v * dx
a.Assemble()

f = LinearForm(V)
f += 1 * v * dx()
f.Assemble()


asmall = InnerProduct (basis, a.mat * basis)
ainvsmall = Matrix(num,num)

f_small = InnerProduct(basis, f.vec)

asmall.Inverse(ainvsmall)
usmall = ainvsmall * f_small

gfu.vec[:] = 0.0

gfu.vec.data = basis * usmall

Draw(gfu, mesh, "gfu")
### big solution

ainv = a.mat.Inverse()
gfu_ex = GridFunction(V)
gfu_ex.vec.data = ainv * f.vec
Draw(gfu_ex, mesh, "gfu_ex")